# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP'

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [6]:
raw = get.get_puma(sample,y1,data_cols)

In [7]:
df = raw.copy()

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(int)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])

In [10]:
df.head()

,GEO_ID,HINCP,NP,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz
4,0400119,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
38,0400120,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40,0400116,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
46,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)].copy()
print(dff.WGTP.sum())
dff['HHSz'] = dff['HHSz'].astype(str)

666238


In [13]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [14]:
pivot = dff.drop(['HINCP','NP','HHSz'],axis=1).copy()
pivot = pivot.groupby(['GEO_ID','AMI_range']).sum().reset_index()
pivot.rename(columns={'WGTP':'hh_e'},inplace=True)

In [16]:
pivot['hh_SE'] = pivot.apply(lambda x: (calc.get_se(x['hh_e'],x[repwts])),axis=1)
pivot['hh_MOE'] = pivot.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
pivot['hh_CV'] = pivot.apply(lambda x: (calc.get_cv(x['hh_e'],x['hh_SE'])),axis=1)
pivot = pivot.drop(columns=repwts)

In [17]:
final = pd.pivot_table(pivot,values=['hh_e','hh_MOE','hh_CV'],index='GEO_ID',\
                      columns=['AMI_range'],aggfunc=np.sum).reset_index()

In [18]:
final

GEO_ID      hh_CV                                            \
AMI_range           30_50_ami 50_80_ami 80_100_ami  o100_ami    u30_ami   
0          0400112   8.865188  7.553053   7.326393  1.850860   8.118226   
1          0400113   7.338969  6.930928   7.840724  2.457191  10.968689   
2          0400114   7.545359  5.549114   6.293865  3.030888   6.988938   
3          0400115   6.917725  5.440686   7.600760  3.491819   7.158460   
4          0400116   6.812999  5.077273   5.385707  2.700971   5.687296   
5          0400117   7.909417  6.530118   7.637212  2.676352   7.211993   
6          0400118   6.493975  5.625607   6.089197  2.830769   5.151657   
7          0400119   8.966435  5.137363   8.788017  3.504936   4.900692   
8          0400120  11.514911  7.768447   9.625971  2.165595   8.625036   
9          0400121   7.892266  5.662385   8.321738  3.221893   6.571832   
10         0400122   7.766873  5.649761   7.251448  4.924724   7.273300   
11         0400123   7.875482  5.369766   8.317253  4.253306   6.814521   
12         0400125   8.016061  5.566549   9.905888  4.005972   7.414963   
13         0400128  10.107936  7.777601   7.429192  2.739979   9.758198   
14         0400129   9.783363  7.581883   7.387722  1.992801   9.795214   

                hh_MOE                                                      \
AMI_range    30_50_ami    50_80_ami   80_100_ami     o100_ami      u30_ami   
0           782.534860  1077.735343   666.331815  2150.989523   749.991699   
1           852.962719   957.268666   744.299242  1826.275099   818.913896   
2          1041.110318  1270.957809   865.022511  1885.151501  1128.115106   
3          1315.796184  1538.809511   874.544877  1617.282254  1009.808778   
4          1161.292728  1279.671764   871.953854  1678.414709   941.096291   
5           838.358600  1467.018341   965.124949  1921.955340   896.167103   
6          1161.214548  1278.278102  1051.924766  1431.218753  1300.092268   
7           934.383458   900.143752   788.088881  1454.439404   856.420952   
8           553.083580  1114.565893   787.695071  1682.684865   945.719141   
9          1034.089936  1550.948974  1228.627446  1531.495795  1029.133614   
10         1148.598324  1164.211140   724.662644  1272.007299  1007.507811   
11         1026.989191  1462.371669  1023.379442  1174.549727  1017.718107   
12         1152.478037  1373.313918  1011.642601  1758.507000  1355.596483   
13         1187.911610   861.849395   844.149345  1960.381189   950.349367   
14          663.174308  1218.695810   649.119480  2038.714691   816.917790   

               hh_e                                        
AMI_range 30_50_ami 50_80_ami 80_100_ami o100_ami u30_ami  
0              3262      5273       3361    42947    3414  
1              4295      5104       3508    27466    2759  
2              5099      8464       5079    22985    5965  
3              7029     10452       4252    17116    5213  
4              6299      9314       5983    22964    6115  
5              3917      8302       4670    26538    4592  
6              6608      8397       6384    18684    9326  
7              3851      6475       3314    15335    6458  
8              1775      5302       3024    28714    4052  
9              4842     10122       5456    17566    5787  
10             5465      7615       3693     9545    5119  
11             4819     10064       4547    10205    5519  
12             5313      9117       3774    16222    6756  
13             4343      4095       4199    26440    3599  
14             2505      5940       3247    37806    3082

In [19]:
final.to_excel('output/hhinc_ami_pumas_21.xlsx')